### 📌 1. Jira의 이슈 불러오기
> 🔗 1. API 호출을 통해 Jira 데이터 가져오기
- Jira의 이슈들을 API와 JQL을 통해 호출

In [9]:
#!/usr/bin/env python3
"""
Jira에서 이슈들을 가져와서 JSONL 형태로 변환하는 스크립트 (임베딩용)
"""

from typing import Dict, List, Optional, Any
import requests
import json
import time
import os
from datetime import datetime
import torch
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Any
import numpy as np

# --- 설정 ---
JIRA_URL = "https://jira.suprema.co.kr"
JIRA_USERNAME =
JIRA_PASSWORD = 
JIRA_AUTH = (JIRA_USERNAME, JIRA_PASSWORD)
# 최종 결과가 저장될 폴더 이름
OUTPUT_FOLDER = "jira_issues_output" # 저장 폴더 이름을 더 명확하게 변경

# 검색할 JQL 쿼리
JQL_QUERY = 'project = "COMMONR" AND issuetype = "Test"'

c:\Program Files\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%pip install -U sentence-transformers

In [5]:
def get_all_jira_issue_keys(jql):
    """JQL로 모든 이슈를 검색하여 이슈 키 리스트를 반환합니다."""
    all_issue_keys = []
    start_at = 0
    max_results = 100

    print(f"JQL로 이슈 검색을 시작합니다: {jql}")

    while True:
        url = f"{JIRA_URL}/rest/api/2/search"
        headers = {"Accept": "application/json"}
        params = {
            'jql': jql,
            'fields': 'key',
            'startAt': start_at,
            'maxResults': max_results
        }

        try:
            response = requests.get(url, headers=headers, params=params, auth=JIRA_AUTH)
            response.raise_for_status()
            data = response.json()
            issues_on_page = data.get('issues', [])
            
            if not issues_on_page:
                break
            
            keys_on_page = [issue['key'] for issue in issues_on_page]
            all_issue_keys.extend(keys_on_page)
            
            start_at += len(issues_on_page)
            print(f"  -> 현재까지 {len(all_issue_keys)} / {data['total']} 개 이슈 키를 가져왔습니다...")
            
        except requests.exceptions.RequestException as e:
            print(f"Jira 이슈 검색 오류: {e}")
            return None
            
    return all_issue_keys

### 📌 2. JSON 파일 열 정제
> 🔗 2. JSON 파일을 임베딩 모델에 임베딩하기 좋은 형식으로 저장
- Jira의 이슈를 가져와서 임베딩 및 langchain에 가공하기 좋은 형식으로 저장

In [6]:
def convert_to_embedding_format(issue_key: str) -> Optional[List[Dict[str, Any]]]:
    """
    하나의 이슈 키에 대한 모든 정보를 Jira API를 통해 가져와서
    임베딩 및 ChromaDB 저장에 최적화된 형식으로 변환합니다.
    
    Args:
        issue_key (str): Jira 이슈 키
        
    Returns:
        Optional[List[Dict[str, Any]]]: 임베딩용 객체들의 리스트
        각 객체는 {'id', 'document', 'metadata'} 구조
    """
    url = f"{JIRA_URL}/rest/api/2/issue/{issue_key}"
    headers = {"Accept": "application/json"}
    
    try:
        response = requests.get(url, headers=headers, auth=JIRA_AUTH)
        response.raise_for_status()
        issue_data = response.json()
        
        if not issue_data:
            return None
        
        # 이슈 데이터 추출
        fields = issue_data.get('fields', {})
        description = fields.get('description', '')
        custom_field_10004 = fields.get('customfield_10004', {})
        steps_list = custom_field_10004.get('steps', [])
        
        embedding_objects = []
        
        # 각 테스트 스텝별로 임베딩 객체 생성
        if isinstance(steps_list, list):
            for item in steps_list:
                if isinstance(item, dict):
                    index = item.get('index', '')
                    step = item.get('step', '')
                    data_item = item.get('data', '')
                    expected_result = item.get('result', '')
                    
                    # page_content: test, step, data, expected result만
                    content_parts = []
                    if step:
                        content_parts.append(f"Test Step: {step}")
                    if data_item:
                        content_parts.append(f"Test Data: {data_item}")
                    if expected_result:
                        content_parts.append(f"Expected Result: {expected_result}")
                    
                    if content_parts:
                        step_object = {
                            "id": f"{issue_key}_step_{index}",
                            "document": "\n".join(content_parts),  # ChromaDB 용어 사용
                            "metadata": {
                                "issue_key": issue_key,
                                "step_index": str(index),  # ChromaDB는 문자열 선호
                                "source": "jira_test_step"  # 데이터 소스 식별용
                            }
                        }
                        embedding_objects.append(step_object)
        
        return embedding_objects
        
    except requests.exceptions.RequestException as e:
        print(f"'{issue_key}'의 정보 조회 중 오류 발생: {e}")
        return None

In [7]:
def json_to_save_file(json_data,issue_key) : 
    #1. 결과를 저장할 폴더 생성
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f" {issue_key} 이슈의 정보를 json으로 저장을 시작합니다.")
    print(f"저장 폴더: '{os.path.abspath(OUTPUT_FOLDER)}'")

    #2. json 파일로 저장장
    try:
        output_filepath = os.path.join(OUTPUT_FOLDER, f"{issue_key}.json")
        with open(output_filepath, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=2)
        return True
    except IOError as e:
        print(f"'{output_filepath}' 파일 저장 중 오류 발생: {e}")
        return False

### 📌 3.JSON 파일 임베딩 모델을 통해 임베딩
> 🔗 JSON 파일을 임베딩 모델을 통해서 임베딩
- e5-multilingual 임베딩 모델을 통해서 임베딩

In [ ]:
def embed_documents_with_e5(embedding_objects: List[Dict[str, Any]], 
                           model_name: str = "intfloat/multilingual-e5-large") -> List[Dict[str, Any]]:
    """
    multilingual-e5 모델을 사용하여 문서들을 임베딩합니다.
    
    Args:
        embedding_objects (List[Dict[str, Any]]): 임베딩할 객체들 
            각 객체는 {'id', 'document', 'metadata'} 구조
        model_name (str): 사용할 E5 모델명 
            - "intfloat/multilingual-e5-large" (권장)
            - "intfloat/multilingual-e5-base" 
            - "intfloat/multilingual-e5-small"
    
    Returns:
        List[Dict[str, Any]]: 임베딩이 추가된 객체들
            각 객체는 {'id', 'document', 'metadata', 'embedding'} 구조
    """
    print(f"E5 모델 로딩 중: {model_name}")
    
    # E5 모델 로드
    model = SentenceTransformer(model_name)
    
    # GPU 사용 가능한지 확인
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    print(f"사용 디바이스: {device}")
    
    # 문서 텍스트 추출
    documents = [obj['document'] for obj in embedding_objects]
    
    print(f"총 {len(documents)}개 문서 임베딩 시작...")
    
    # E5 모델의 경우 쿼리용 접두사 추가 (검색 시에는 "query: " 사용)
    # 문서 임베딩시에는 접두사 없이 사용하거나 "passage: " 사용
    prefixed_documents = [f"passage: {doc}" for doc in documents]
    
    # 배치 임베딩 (메모리 효율성을 위해 배치 크기 조정 가능)
    embeddings = model.encode(
        prefixed_documents,
        batch_size=8,  # 메모리에 따라 조정
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True  # 코사인 유사도 최적화
    )
    
    print(f"임베딩 완료! 벡터 차원: {embeddings.shape[1]}")
    
    # 원본 객체에 임베딩 추가
    result_objects = []
    for i, obj in enumerate(embedding_objects):
        embedded_obj = obj.copy()
        embedded_obj['embedding'] = embeddings[i].tolist()  # numpy array를 list로 변환
        result_objects.append(embedded_obj)
    
    return result_objects
    

### 📌 4. 벡터 DB를 통해서 저장
> 🔗 해당 임베딩한 것을 벡터 DB를 통해서 저장
- Chroma DB를 사용해서 임베딩 값 저장

In [ ]:
def main():
    """메인 실행 함수"""
    print("=== Jira to 임베딩 형식 변환 ===")
    print(f"Jira URL: {JIRA_URL}")
    print(f"사용자: {JIRA_USERNAME}")
    print(f"JQL 쿼리: {JQL_QUERY}")
    print()

    # 1. 이슈 키 목록 가져오기
    issue_keys = get_all_jira_issue_keys(JQL_QUERY)
    
    if not issue_keys:
        print("검색된 이슈가 없습니다.")
        return
    
    print(f"총 {len(issue_keys)}개의 이슈를 찾았습니다.")
    print()
    
    # 2. 모든 이슈 데이터를 임베딩 형태로 변환
    all_embedding_data = []
    successful_count = 0
    
    print("이슈 데이터를 가져오고 임베딩 형식으로 변환하는 중...")
    
    for i, issue_key in enumerate(issue_keys, 1):
        print(f"[{i}/{len(issue_keys)}] {issue_key} 처리 중...")
        
        # 임베딩용 형태로 변환
        embedding_objects = convert_to_embedding_format(issue_key)
        if embedding_objects:
            all_embedding_data.extend(embedding_objects)  # 리스트 확장
            # 개별 이슈별로 JSON 파일 저장
            json_to_save_file(embedding_objects, issue_key)
            successful_count += 1
        
        # API 호출 제한 고려
        time.sleep(0.1)
    
    print(f"\n성공적으로 처리된 이슈: {successful_count}/{len(issue_keys)}")
    
    # 3. 임베딩 데이터 출력 (처음 3개만 샘플로)
    print("\n=== 임베딩 데이터 샘플 ===")
    for i, item in enumerate(all_embedding_data[:3]):
        print(f"샘플 {i+1}:")
        print(f"ID: {item['id']}")
        print(f"Document: {item['document'][:100]}...")
        print(f"Metadata: {item['metadata']}")
        print("-" * 50)
    
    # 4. 통계
    print(f"\n=== 통계 ===")
    print(f"총 {len(all_embedding_data)}개의 임베딩 객체 생성")
    print(f"이슈당 평균 {len(all_embedding_data)/len(issue_keys):.1f}개의 테스트 스텝")
    
    # 평균 문서 길이
    if all_embedding_data:
        avg_doc_length = sum(len(item['document']) for item in all_embedding_data) / len(all_embedding_data)
        print(f"평균 문서 길이: {avg_doc_length:.0f} 문자")

    #5. 임베딩 실행
    embed_documents_with_e5(all_embedding_data)
    return

if __name__ == "__main__":
    main()
    print("스크립트 실행 중...")

=== Jira to 임베딩 형식 변환 ===
Jira URL: https://jira.suprema.co.kr
사용자: dhwoo
JQL 쿼리: project = "COMMONR" AND issuetype = "Test"

JQL로 이슈 검색을 시작합니다: project = "COMMONR" AND issuetype = "Test"
  -> 현재까지 100 / 186 개 이슈 키를 가져왔습니다...
  -> 현재까지 186 / 186 개 이슈 키를 가져왔습니다...
총 186개의 이슈를 찾았습니다.

이슈 데이터를 가져오고 임베딩 형식으로 변환하는 중...
[1/186] COMMONR-380 처리 중...
 COMMONR-380 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[2/186] COMMONR-379 처리 중...
 COMMONR-379 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[3/186] COMMONR-378 처리 중...
 COMMONR-378 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[4/186] COMMONR-377 처리 중...
 COMMONR-377 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[5/186] COMMONR-376 처리 중...
 COMMONR-376 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[6/186] COMMONR-375 처리 중...
 COMMO